## Model Checkpoint to Save Weights
You've now trained a neural network or two, but how can we actually make use of the "magic" of the model?  If the problem we're trying to solve is classifying a breast cancer tumor as benign or malignant, what do we do after we get the pretty >$98\%$ accuracy?  The answer is to save the weights and model into a separate file to access later!

**NOTE** In order to save weights, you must have the Python package h5py installed

In [10]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.utils import shuffle

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

from keras_tqdm import TQDMNotebookCallback

In [2]:
# Load up data
data = np.array(pd.read_table(
    "https://www.cs.mtsu.edu/~jphillips/courses/CSCI4850-5850/public/WDBC.txt",
    delim_whitespace=True,
    header=None))

# Separate data into features and labels
nb_features = 30
X = data[:, :nb_features]
Y = data[:, nb_features]
X, Y = shuffle(X,Y)

Note that for the purposes of this example, we will skip splitting the data into training and testing sets.

In [3]:
# create a simple neural network
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
            optimizer=Adam(lr=1e-2), 
            metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               3968      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 20,609
Trainable params: 20,609
Non-trainable params: 0
_________________________________________________________________


Up until now, everything has been the same as we have done.  The only change to make is to add a callback, and then pass the callback to the fit() function.

In [16]:
# create a directory to hold the weights
weight_dir = "weights"
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)

# the filename will be populated at the end of each epoch, when the callback is called
# values enclosed in curly braces will be substitued by Keras
weight_filename = "wdbc-weights_epoch-{epoch:04d}_acc-{acc:.2f}.hdf5"
weight_filepath = os.path.join(weight_dir, weight_filename)

checkpoint = ModelCheckpoint(weight_filepath, 
                         monitor='acc', 
                         verbose=0,)

# the argument passed into the fit() function must be a list
# if using multiple callbacks, you can use the .append() function or initialize them inline
callbacks_list = [checkpoint,TQDMNotebookCallback()]

There are many more arguments which can be passed to ModelCheckpoint(), and you can see them here: https://keras.io/callbacks/#modelcheckpoint

The key parameters in practice are the name of the file itself, and whether or not to save only the weights or the weights alongside the model architecture.  This allows you to keep track of all the models you make and access them easily.

In [13]:
history_xor = model.fit(X, Y,
                       batch_size=64,
                       epochs=1000,
                       verbose=0,
                       callbacks=callbacks_list)

A Jupyter Widget

A Jupyter Widget

KeyboardInterrupt: 

Now we can check and see what's inside our weights folder!

In [14]:
weight_files = os.listdir(weight_dir)
weight_files.sort()
weight_files

['wdbc-weights_epoch-0001_acc-0.96.hdf5',
 'wdbc-weights_epoch-0001_acc-0.98.hdf5',
 'wdbc-weights_epoch-0002_acc-0.96.hdf5',
 'wdbc-weights_epoch-0002_acc-0.97.hdf5',
 'wdbc-weights_epoch-0003_acc-0.97.hdf5',
 'wdbc-weights_epoch-0003_acc-0.98.hdf5',
 'wdbc-weights_epoch-0004_acc-0.97.hdf5',
 'wdbc-weights_epoch-0004_acc-0.98.hdf5',
 'wdbc-weights_epoch-0005_acc-0.96.hdf5',
 'wdbc-weights_epoch-0005_acc-0.98.hdf5',
 'wdbc-weights_epoch-0006_acc-0.96.hdf5',
 'wdbc-weights_epoch-0006_acc-0.97.hdf5',
 'wdbc-weights_epoch-0007_acc-0.96.hdf5',
 'wdbc-weights_epoch-0007_acc-0.97.hdf5',
 'wdbc-weights_epoch-0008_acc-0.97.hdf5',
 'wdbc-weights_epoch-0008_acc-0.98.hdf5',
 'wdbc-weights_epoch-0009_acc-0.97.hdf5',
 'wdbc-weights_epoch-0009_acc-0.98.hdf5',
 'wdbc-weights_epoch-0010_acc-0.97.hdf5',
 'wdbc-weights_epoch-0011_acc-0.99.hdf5',
 'wdbc-weights_epoch-0012_acc-0.99.hdf5',
 'wdbc-weights_epoch-0013_acc-0.99.hdf5',
 'wdbc-weights_epoch-0014_acc-0.98.hdf5',
 'wdbc-weights_epoch-0015_acc-0.99

We now have access to all of our weights! We can load a model up and test it like so:

In [18]:
weight_dir

'weights'

In [17]:
# load the most recent file (last element in the sorted list)
weight_file_to_load = os.path.join(weight_dir, weight_files[-1])
model_2 = load_model(weight_file_to_load)
model_2.summary()

KeyError: "Can't open attribute (Can't locate attribute: 'weight_names')"

In [8]:
print("Loaded model's evaluation: {:.2f}".format(100 * model_2.evaluate(X,Y)[1]))

568/568 [==============================] - 0s 159us/step
Loaded model's evaluation: 97.18


These model files can be loaded up by any script running Keras to start making use of your classifications right away.